In [2]:
import numpy as np # linear algebra
import pandas as pd # data manipulation and analysis
import matplotlib.pyplot as plt # data visualization
import seaborn as sns # data visualization
sns.set_style('whitegrid') # set style for visualization
import warnings # ignore warnings
warnings.filterwarnings('ignore')


!kaggle datasets download -d brendan45774/test-file -p ./data --unzip
titanic_data = pd.read_csv('./data/tested.csv')
titanic_data.head(6)

Dataset URL: https://www.kaggle.com/datasets/brendan45774/test-file
License(s): CC0-1.0




  0%|          | 0.00/11.2k [00:00<?, ?B/s]
100%|██████████| 11.2k/11.2k [00:00<00:00, 3.01MB/s]


,PassengerId,Survived,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked
0,892,0,3,"Kelly, Mr. James",male,34.5,0,0,330911,7.8292,NaN,Q
1,893,1,3,"Wilkes, Mrs. James (Ellen Needs)",female,47.0,1,0,363272,7.0000,NaN,S
2,894,0,2,"Myles, Mr. Thomas Francis",male,62.0,0,0,240276,9.6875,NaN,Q
3,895,0,3,"Wirz, Mr. Albert",male,27.0,0,0,315154,8.6625,NaN,S
4,896,1,3,"Hirvonen, Mrs. Alexander (Helga E Lindqvist)",female,22.0,1,1,3101298,12.2875,NaN,S
5,897,0,3,"Svensson, Mr. Johan Cervin",male,14.0,0,0,7538,9.2250,NaN,S


In [3]:
titanic_data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 418 entries, 0 to 417
Data columns (total 12 columns):
 #   Column       Non-Null Count  Dtype  
---  ------       --------------  -----  
 0   PassengerId  418 non-null    int64  
 1   Survived     418 non-null    int64  
 2   Pclass       418 non-null    int64  
 3   Name         418 non-null    object 
 4   Sex          418 non-null    object 
 5   Age          332 non-null    float64
 6   SibSp        418 non-null    int64  
 7   Parch        418 non-null    int64  
 8   Ticket       418 non-null    object 
 9   Fare         417 non-null    float64
 10  Cabin        91 non-null     object 
 11  Embarked     418 non-null    object 
dtypes: float64(2), int64(5), object(5)
memory usage: 39.3+ KB


In [4]:
titanic_data.describe()

,PassengerId,Survived,Pclass,Age,SibSp,Parch,Fare
count,418.000000,418.000000,418.000000,332.000000,418.000000,418.000000,417.000000
mean,1100.500000,0.363636,2.265550,30.272590,0.447368,0.392344,35.627188
std,120.810458,0.481622,0.841838,14.181209,0.896760,0.981429,55.907576
min,892.000000,0.000000,1.000000,0.170000,0.000000,0.000000,0.000000
25%,996.250000,0.000000,1.000000,21.000000,0.000000,0.000000,7.895800
50%,1100.500000,0.000000,3.000000,27.000000,0.000000,0.000000,14.454200
75%,1204.750000,1.000000,3.000000,39.000000,1.000000,0.000000,31.500000
max,1309.000000,1.000000,3.000000,76.000000,8.000000,9.000000,512.329200


In [5]:
age_mean_men = titanic_data['Age'].mean()
print(age_mean_men)

30.272590361445783


In [6]:
# Simulate extra demographic data (for integration example)
demographics = titanic_data[['PassengerId']].copy()
# Create a new column in the demographics pd, column='Has_Pets' and is randomly assigned either True or False
demographics['Has_Pets'] = np.random.choice([True, False], size=len(demographics))
# print(demographics)

# Merge
titanic_data = pd.merge(titanic_data, demographics, on='PassengerId', how='left')  # Notice when merging 2 dataframes, the column name has to be exactly the same
titanic_data[['PassengerId', 'Has_Pets']].head()

# The LEFT JOIN keyword returns all records from the left table (table1), and the matching records from the right table (table2).


,PassengerId,Has_Pets
0,892,True
1,893,True
2,894,False
3,895,False
4,896,False


In [7]:
duplicate_mask = titanic_data.duplicated()
print(duplicate_mask)

0      False
1      False
2      False
3      False
4      False
       ...  
413    False
414    False
415    False
416    False
417    False
Length: 418, dtype: bool


In [8]:
## 2.Add a duplicate of the first row
titanic_data = pd.concat([titanic_data, titanic_data.iloc[[0]]], ignore_index=True)

# Check for duplicate rows
duplicate_mask = titanic_data.duplicated()

# Print number of duplicate rows
print("Number of duplicate rows:", duplicate_mask.sum())

# Optionally, view the duplicated rows
print(titanic_data[duplicate_mask])


Number of duplicate rows: 1
     PassengerId  Survived  Pclass              Name   Sex   Age  SibSp  \
418          892         0       3  Kelly, Mr. James  male  34.5      0   

     Parch  Ticket    Fare Cabin Embarked  Has_Pets  
418      0  330911  7.8292   NaN        Q      True  


In [9]:
# Remove Duplicates
titanic_data = titanic_data.drop_duplicates()
print("Remaining duplicates:", titanic_data.duplicated().sum())

Remaining duplicates: 0


In [10]:
# Remove Irrelevant Columns
# It is safer to copy the original data and drop the irrelevant columns from the copy
titanic_data = titanic_data.drop(['Cabin', 'Ticket'], axis=1, errors='ignore')
print("Columns after removal:", titanic_data.columns)

Columns after removal: Index(['PassengerId', 'Survived', 'Pclass', 'Name', 'Sex', 'Age', 'SibSp',
       'Parch', 'Fare', 'Embarked', 'Has_Pets'],
      dtype='object')


In [11]:
# Fixing Structural Errors - simulate 'Date' column
titanic_data['Date'] = pd.date_range(start='1/1/1912', periods=len(titanic_data), freq='D')
titanic_data['Date'] = pd.to_datetime(titanic_data['Date'])
print(titanic_data[['Date']].head())

        Date
0 1912-01-01
1 1912-01-02
2 1912-01-03
3 1912-01-04
4 1912-01-05


In [12]:
import pandas as pd
# Check for missing values in a DataFrame
missing_data = titanic_data.isnull()
print(missing_data.head())

   PassengerId  Survived  Pclass   Name    Sex    Age  SibSp  Parch   Fare  \
0        False     False   False  False  False  False  False  False  False   
1        False     False   False  False  False  False  False  False  False   
2        False     False   False  False  False  False  False  False  False   
3        False     False   False  False  False  False  False  False  False   
4        False     False   False  False  False  False  False  False  False   

   Embarked  Has_Pets   Date  
0     False     False  False  
1     False     False  False  
2     False     False  False  
3     False     False  False  
4     False     False  False  


In [13]:
# Identify Missing
print("Missing values per column:")
print(titanic_data.isnull().sum())

Missing values per column:
PassengerId     0
Survived        0
Pclass          0
Name            0
Sex             0
Age            86
SibSp           0
Parch           0
Fare            1
Embarked        0
Has_Pets        0
Date            0
dtype: int64


In [14]:
# Make a copy of the dataset to not permanently damage the original set
titanic_filled = titanic_data.copy()

# SYNTAX : UPDATED_COLUMN = DESIRED COLUMN . fill null values (mean of the 'Age' column of titanic dataset)
# This fills all the NULL values in the 'Age' column with the average age of the entire set
titanic_filled['Age'] = titanic_filled['Age'].fillna(titanic_filled['Age'].mean())

# Check if this worked
print("Missing values per column:")
print(titanic_filled.isnull().sum())

Missing values per column:
PassengerId    0
Survived       0
Pclass         0
Name           0
Sex            0
Age            0
SibSp          0
Parch          0
Fare           1
Embarked       0
Has_Pets       0
Date           0
dtype: int64


KNNImputer is a technique for filling in missing values based on the values of the k-nearest neighbors (rows that are similar in other features). It’s more intelligent than filling with mean or median, because it accounts for relationships between features.

✅ Create the imputer object.

n_neighbors=3 means for each missing value, the algorithm will:

Find the 3 rows most similar to the one with missing data (based on other numeric columns).

Use their values to compute the average and fill in the missing value.


 Filter only numeric columns, because KNNImputer can only work with numbers.
This ensures we don’t feed in strings or categories like 'Sex' or 'Embarked'.


Perform the imputation:

fit_transform() does two things:

Fit: Learns how to find neighbors based on available (non-missing) data.

Transform: Fills in the missing values using neighbor-based averages.

The result is a NumPy array with all missing values filled.

✅ Convert the result back to a DataFrame, keeping the original column names.

✅ Just to check: Print the first few rows of the cleaned, imputed data.

In [16]:
from sklearn.impute import KNNImputer

imputer = KNNImputer(n_neighbors=3)
df_numeric = titanic_data.select_dtypes(include=['float64', 'int64'])
imputed_array = imputer.fit_transform(df_numeric)
df_imputed = pd.DataFrame(imputed_array, columns=df_numeric.columns)
print(df_imputed.head())

   PassengerId  Survived  Pclass   Age  SibSp  Parch     Fare
0        892.0       0.0     3.0  34.5    0.0    0.0   7.8292
1        893.0       1.0     3.0  47.0    1.0    0.0   7.0000
2        894.0       0.0     2.0  62.0    0.0    0.0   9.6875
3        895.0       0.0     3.0  27.0    0.0    0.0   8.6625
4        896.0       1.0     3.0  22.0    1.0    1.0  12.2875


In [17]:
# Identify and handle outliers in the 'Fare' column
Q1 = titanic_data['Fare'].quantile(0.25)
Q3 = titanic_data['Fare'].quantile(0.75)
IQR = Q3 - Q1

# Determine the upper/lower bounds for outlier values
lower_bound = Q1 - 1.5 * IQR
upper_bound = Q3 + 1.5 * IQR

# Present only values in the Inter-Quartile-Range (IQR)
filtered_fare = titanic_data[(titanic_data['Fare'] >= lower_bound) & (titanic_data['Fare'] <= upper_bound)]
print("Original shape:", titanic_data.shape)
print("Shape after removing fare outliers:", filtered_fare.shape)

outlier_fare = titanic_data[(titanic_data['Fare'] < lower_bound) | (titanic_data['Fare'] > upper_bound)]
print("Outliers:", outlier_fare.shape)

Original shape: (418, 12)
Shape after removing fare outliers: (362, 12)
Outliers: (55, 12)


What’s going on?
Q1 (25th percentile) is the value below which 25% of the data falls.

Q3 (75th percentile) is the value below which 75% of the data falls.

IQR (Interquartile Range) is the difference between Q3 and Q1:

IQR
=
𝑄
3
−
𝑄
1
IQR=Q3−Q1
It measures the spread of the middle 50% of the data.


What’s going on?
These formulas define the acceptable range for Age.

Any value outside this range is considered an outlier.

Lower Bound
=
𝑄
1
−
1.5
×
𝐼
𝑄
𝑅
Lower Bound=Q1−1.5×IQR
Upper Bound
=
𝑄
3
+
1.5
×
𝐼
𝑄
𝑅
Upper Bound=Q3+1.5×IQR
This "1.5 * IQR" rule is a common statistical convention:

Too far below = unusually small → outlier

Too far above = unusually large → outlier

In [18]:
# 1. Q1 and Q3
Q1 = titanic_data['Age'].quantile(0.25)
Q3 = titanic_data['Age'].quantile(0.75)

# 2. Bounds
IQR = Q3 - Q1
lower_bound = Q1 - 1.5 * IQR
upper_bound = Q3 + 1.5 * IQR

# 3. Count Outliers
quartile_age = titanic_data[(titanic_data['Age'] >= lower_bound) & (titanic_data['Age'] <= upper_bound)]
print("Original Shape:", titanic_data.shape)
print("IQR Shape:", quartile_age.shape)

outliers = titanic_data[(titanic_data['Age'] < lower_bound) | (titanic_data['Age'] > upper_bound)]
print("Outliers:", outliers.shape)  # Doesn't give the complement of the IQR (891 - 703) due to the null values in titanic_data['Age']

Original Shape: (418, 12)
IQR Shape: (330, 12)
Outliers: (2, 12)


In [1]:
import pandas as pd
import numpy as np
from sklearn.preprocessing import MinMaxScaler
from sklearn.decomposition import PCA

# Create a sample dataset
np.random.seed(42)
data = {
    'CustomerID': range(1, 11),
    'Age': np.random.randint(18, 70, 10),
    'Annual_Income': np.random.randint(30000, 120000, 10),
    'Spending_Score': np.random.randint(1, 100, 10),
    'Purchase_Amount': np.random.randint(100, 1000, 10),
    'Purchase_Date': pd.date_range(start='2024-01-01', periods=10, freq='15D')
}
df = pd.DataFrame(data)
df.head()

,CustomerID,Age,Annual_Income,Spending_Score,Purchase_Amount,Purchase_Date
0,1,56,67194,30,352,2024-01-01
1,2,69,117498,38,847,2024-01-16
2,3,46,74131,2,956,2024-01-31
3,4,32,90263,64,660,2024-02-15
4,5,60,46023,60,574,2024-03-01


✅ 1. Data Normalization
What it is: Rescaling features to a standard range (typically 0 to 1) so that no feature dominates others due to its scale.

Why it's important: Algorithms like KNN, clustering, and PCA are sensitive to scale — normalization helps them treat all features equally.

In the example: We normalized Age, Annual_Income, and Spending_Score using MinMaxScaler to create new columns ending in _norm.



In [2]:

# 1. Data Normalization (Min-Max Scaling)
scaler = MinMaxScaler()
df[['Age_norm', 'Annual_Income_norm', 'Spending_Score_norm']] = scaler.fit_transform(
    df[['Age', 'Annual_Income', 'Spending_Score']]
)
df


,CustomerID,Age,Annual_Income,Spending_Score,Purchase_Amount,Purchase_Date,Age_norm,Annual_Income_norm,Spending_Score_norm
0,1,56,67194,30,352,2024-01-01,0.704545,0.419986,0.378378
1,2,69,117498,38,847,2024-01-16,1.000000,1.000000,0.486486
2,3,46,74131,2,956,2024-01-31,0.477273,0.499971,0.000000
3,4,32,90263,64,660,2024-02-15,0.159091,0.685976,0.837838
4,5,60,46023,60,574,2024-03-01,0.795455,0.175881,0.783784
5,6,25,71090,21,158,2024-03-16,0.000000,0.464908,0.256757
6,7,38,97221,33,610,2024-03-31,0.295455,0.766203,0.418919
7,8,56,94820,76,781,2024-04-15,0.704545,0.738519,1.000000
8,9,36,30769,58,575,2024-04-30,0.250000,0.000000,0.756757
9,10,40,89735,22,799,2024-05-15,0.340909,0.679888,0.270270


✅ 2. Data Reduction using PCA
What it is: Reduces the number of columns (features) while keeping most of the information.

Why it's important: Makes data easier to visualize and can improve performance in high-dimensional datasets.

In the example: We used PCA to reduce the normalized features to two components (PCA1, PCA2) — these represent the most meaningful variance in the data.

In [ ]:
# 2. Data Reduction using PCA
pca = PCA(n_components=2)
pca_features = pca.fit_transform(df[['Age_norm', 'Annual_Income_norm', 'Spending_Score_norm']])
df[['PCA1', 'PCA2']] = pca_features
df

In [ ]:
print(pca.explained_variance_ratio_)


In [ ]:
import matplotlib.pyplot as plt

# Plotting PCA1 vs PCA2 with labels
plt.figure(figsize=(8, 6))
plt.scatter(df['PCA1'], df['PCA2'], c='blue', edgecolors='k', s=80)
plt.title('PCA: Data Projection onto 2 Principal Components')
plt.xlabel('Principal Component 1 (PCA1)')
plt.ylabel('Principal Component 2 (PCA2)')
plt.grid(True)
plt.tight_layout()
plt.show()


✅ 3. Data Aggregation
What it is: Summarizing data — typically grouping by a category or time period and applying aggregation functions (e.g., sum, mean).

Why it's important: Allows simplification of raw data for trend analysis, reporting, or dashboard creation.

In the example: We aggregated the Purchase_Amount by month using .groupby() to see how total purchases vary over time.

In [ ]:

# 3. Data Aggregation - Group by Month and calculate total Purchase_Amount
df['Month'] = df['Purchase_Date'].dt.to_period('M')
monthly_agg = df.groupby(['Month', 'CustomerID']).agg(
    Purchase_Amount_min=('Purchase_Amount', 'min'),
    Purchase_Amount_max=('Purchase_Amount', 'max')
).reset_index()

print(monthly_agg)


✅ 1. Data Normalization
What it is: Rescaling features to a standard range (typically 0 to 1) so that no feature dominates others due to its scale.

Why it's important: Algorithms like KNN, clustering, and PCA are sensitive to scale — normalization helps them treat all features equally.

In the example: We normalized Age, Annual_Income, and Spending_Score using MinMaxScaler to create new columns ending in _norm.



In [ ]:
from google.colab import files
files.upload()
 # Upload kaggle.json# ✅ 2. Import libraries and load data
import pandas as pd
import numpy as np

weather_data = pd.read_csv('weatherHistory.csv')
print(weather_data.head(6))



In [ ]:
print(weather_data.info())

In [ ]:
print(weather_data.describe())

In [ ]:
weather_data.head()

In [ ]:
# 1. Data Normalization (Min-Max Scaling)
scaler = MinMaxScaler()
weather_data[['Temperature (C)_norm', 'Wind Speed (km/h)_norm', 'Pressure (millibars)_norm']] = scaler.fit_transform(
    weather_data[['Temperature (C)', 'Wind Speed (km/h)', 'Pressure (millibars)']]
)
weather_data

In [ ]:
# 2. Data Reduction using PCA
pca = PCA(n_components=2)
pca_features = pca.fit_transform(weather_data[['Temperature (C)_norm', 'Wind Speed (km/h)_norm', 'Pressure (millibars)_norm']])
weather_data[['PCA1', 'PCA2']] = pca_features
weather_data

In [ ]:
import matplotlib.pyplot as plt
# Plotting PCA1 vs PCA2 with labels
plt.figure(figsize=(8, 6))
plt.scatter(weather_data['PCA1'], weather_data['PCA2'], c='blue', edgecolors='k', s=80)
plt.title('PCA: Data Projection onto 2 Principal Components')
plt.xlabel('Principal Component 1 (PCA1)')
plt.ylabel('Principal Component 2 (PCA2)')
plt.grid(True)
plt.tight_layout()
plt.show()

In [ ]:
 # 3. Data Aggregation - Group by Month and calculate the average Temperature (C)
weather_data['Formatted Date'] = pd.to_datetime(weather_data['Formatted Date'], utc=True)
weather_data['Month'] = weather_data['Formatted Date'].dt.to_period('M')

daily_agg = weather_data.groupby('Month').agg({'Temperature (C)': 'mean'})
daily_agg


In [ ]:
import pandas as pd
from sklearn.preprocessing import MinMaxScaler
from sklearn.decomposition import PCA


# 1. Data Normalization
scaler = MinMaxScaler()
numerical_cols = weather_data.select_dtypes(include='number').columns
weather_data_normalized = scaler.fit_transform(weather_data[numerical_cols])
weather_data[[col + '_norm' for col in numerical_cols]] = pd.DataFrame(weather_data_normalized, columns=[col + '_norm' for col in numerical_cols])
weather_data

# 2. PCA for dimensionality reduction
pca = PCA(n_components=2)
pca_result = pca.fit_transform(weather_data[[col + '_norm' for col in numerical_cols]])
weather_data[['PCA1', 'PCA2']] = pca_result

# 3. Aggregation: Average temperature and humidity by summary
weather_agg = weather_data.groupby('Daily Summary').agg(
    Avg_Temp=('Apparent Temperature (C)', 'mean'),
    Avg_Humidity=('Humidity', 'mean')
).reset_index()

